# Denoising Autoencoder

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Helpers/Base.ipynb");

In [ ]:
# Parameters
model = "autoencoder"
task = "random"
content = "implicit"
input_alphas = String[]
residual_alphas = String[]
outdir = "$task/$content/NeuralImplicitItemCFUntuned"
tune_hyperparams = false;

In [ ]:
hyp = create_hyperparams(model, content, task, residual_alphas, input_alphas)

In [ ]:
# G = hyp
# m = build_model(rng = rng) |> device
# opt = Optimisers.setup(opt_spec, m)
# train_epoch!(m, opt; split = "training", rng = rng)
# split_loss(m, "validation")

In [ ]:
train_alpha(
    hyp,
    outdir;
    tune_hyperparams=tune_hyperparams
)